In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format)
import numpy as np

In [ ]:
ticker="slv"
df=pd.read_csv(f"data/etf/jan2025/selected/{ticker}_20250103_20250131.csv",engine="pyarrow", sep = ',')
df["Date-Time"] = df["Date-Time"].dt.tz_convert("America/New_York")
df.head(3)
df.describe()

In [ ]:
pd.DataFrame(pd.to_datetime(df["Date-Time"]).dt.date.unique())

In [ ]:
df["Date"] = pd.to_datetime(df["Date-Time"]).dt.date
grouped = df.groupby("Date").size().reset_index(name="Count")


In [ ]:
# Filter only regular market hours (09:30–16:00 Eastern)
df_market = df[
    (df["Date-Time"].dt.time >= pd.to_datetime("09:40:00").time()) &
    (df["Date-Time"].dt.time <= pd.to_datetime("15:50:00").time())
]

In [ ]:
base_cols = ["#RIC", "Date-Time"]
#base_cols = ["#RIC"]
# Dynamically build list for L1–L10
levels = []
for i in range(1, 11):  # L1–L10 inclusive
    levels += [
        f"L{i}-BidPrice", f"L{i}-BidSize",
        f"L{i}-AskPrice", f"L{i}-AskSize"
    ]


In [ ]:
df_market_l10 = df_market[base_cols + levels]

In [ ]:
df_market_l10.head(10)
df_market_l10.describe()

In [ ]:
price_cols = df_market_l10.filter(regex=r'^L(?:10|[1-9])-(?:Bid|Ask)Price$').columns

# Keep only rows where ANY price column changed vs the previous row
changed = df_market_l10[price_cols].ne(df_market_l10[price_cols].shift()).any(axis=1)
df_filtered = df_market_l10[changed].copy()

In [ ]:
print(len(df_filtered))
print(len(df_market_l10))

In [ ]:
df_market_l10=df_filtered

In [ ]:
df_market_l10["ms_timestamp"] = df_market_l10["Date-Time"].dt.floor("ms")
df_market_l10 = (
   df_market_l10.drop_duplicates(subset=["#RIC", "ms_timestamp"], keep="last")
     .drop(columns="ms_timestamp")
)
df_market_l10=df_market_l10.dropna()

In [ ]:
df_market_l10.head(10)
df_market_l10.describe()

In [ ]:
#df_market_l10.drop("mid_price",axis=1, inplace=True)
#df_market_l10.drop("abs.spread",axis=1, inplace=True)
#df_market_l10.drop("rel.spread",axis=1, inplace=True)

In [ ]:
df_market_l10.insert(
            2, "mid_price", (df_market_l10["L1-BidPrice"] + df_market_l10["L1-AskPrice"]) / 2
        )  # Add the mid-price column to the orderbook dataframe.
df_market_l10.insert(
            2, "abs.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])
        )
df_market_l10.insert(
            2, "rel.spread", (df_market_l10["L1-AskPrice"] - df_market_l10["L1-BidPrice"])/df_market_l10["mid_price"]
        )

In [ ]:
df_market_l10.head(10)

In [ ]:
print(sum(round(df_market_l10["abs.spread"],2)==float(0.01)))
tick_size=sum(round(df_market_l10["abs.spread"],2)==float(0.01))/df_market_l10.size
print(tick_size)

In [ ]:
df_market_l10.describe()

In [ ]:
df_market_l10

In [ ]:
df_market_l10[~df_market_l10.isna().any(axis=1)].to_csv(f"data/etf/jan2025/cleaned/{ticker}_cleaned_jan2025.csv", index=False)

In [ ]:
df_market_l10.describe()